In [3]:
import pandas as pd
import sqlalchemy
import pandas_ta
from sklearn.preprocessing import normalize, StandardScaler
from sklearn.metrics import confusion_matrix
def ticker_preprocess(result_set):
    result_set.set_index('Date',inplace=True)
    result_set.ta.vwma(close='Adj Close', append=True, length=5)
    result_set.ta.vwma(close='Adj Close', append=True, length=20)
    result_set.ta.vwma(close='Adj Close', append=True, length=90)
    result_set.ta.mfi(close='Adj Close', append=True)
    result_set.ta.macd(append=True)
    result_set.ta.rsi(append=True)
    result_set.ta.bbands(append=True)
    result_set.ta.obv(append=True)
    result_set.ta.sma(length=20, append=True)
    result_set.ta.ema(length=50, append=True)
    result_set.ta.stoch(append=True)
    result_set.ta.adx(append=True)
    result_set.ta.ad(append=True)
    result_set.ta.stdev(append=True)
    
    result_set['Y_CLOSE5_MAX'] = result_set['Adj Close'].rolling(5).max().shift(-5)
    result_set['Y_CLOSE5_MIN'] = result_set['Adj Close'].rolling(5).min().shift(-5)
    
    result_set['Volume_5'] = result_set['Volume'].rolling(5).mean()
    result_set['Volume_20'] = result_set['Volume'].rolling(20).mean()
    result_set['Volume_90'] = result_set['Volume'].rolling(90).mean()

    result_set['delta_p5'] = result_set['Adj Close'] - result_set['VWMA_5']
    result_set['delta_p20'] = result_set['Adj Close'] - result_set['VWMA_20']
    result_set['delta_p90'] = result_set['Adj Close'] - result_set['VWMA_90']

    result_set['delta_v5'] = result_set['Volume'] - result_set['Volume_5']
    result_set['delta_v20'] = result_set['Volume'] - result_set['Volume_20']
    result_set['delta_v90'] = result_set['Volume'] - result_set['Volume_90']
    
    result_set['PV_DELTA_R5'] = result_set['delta_p5'] /result_set['delta_v5'] 
    result_set['PV_DELTA_R20'] = result_set['delta_p20'] /result_set['delta_v20'] 
    result_set['PV_DELTA_R90'] = result_set['delta_p90'] /result_set['delta_v90'] 

    result_set['PV_DELTA_R20_PRIOR1'] = result_set['PV_DELTA_R20'].shift(1)
    result_set['PV_DELTA_R20_PRIOR2'] = result_set['PV_DELTA_R20'].shift(2)
    result_set['PV_DELTA_R20_PRIOR3'] = result_set['PV_DELTA_R20'].shift(3)
    result_set['PV_DELTA_R20_PRIOR4'] = result_set['PV_DELTA_R20'].shift(4)
    result_set['PV_DELTA_R20_PRIOR5'] = result_set['PV_DELTA_R20'].shift(5)
    
    engine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
    result_set.to_sql('STOCK_D_X', engine, if_exists='append')
    engine.dispose()
    


In [4]:
query = '''
SELECT * FROM STOCK_D_BASIC
'''
sqlengine = sqlalchemy.create_engine('sqlite:///D:\\sqlite\\data\\finance.db')
result_set = pd.read_sql_query(query, sqlengine.connect())
grouped = result_set.groupby(['ticker'])
sqlengine.dispose()
for name, group in grouped:
    group = ticker_preprocess(group)


  
